# Week 4: Advanced SQL for Marketing Analytics

**Goal:** Master advanced SQL techniques for complex marketing measurement and attribution.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- Advanced window functions (LAG, LEAD, NTILE, PERCENT_RANK)
- Common Table Expressions (CTEs) for readable queries
- Temporary tables and query optimization
- Date/time operations for cohort analysis
- String manipulation for campaign parsing
- Advanced joins and set operations
- Complex marketing analytics queries

**Why This Matters:**
As a Marketing Measurement Partner, you'll tackle complex questions:
- Which campaigns drove the highest customer lifetime value?
- How do customers behave across their first 90 days?
- What's the optimal budget distribution across channels?
- How has campaign efficiency trended month-over-month?
- Which touchpoints contribute most to conversions?

Advanced SQL lets you answer these questions efficiently and elegantly.

---

## Setup: Enhanced Marketing Database

We'll create an enhanced database with more realistic data for advanced analysis.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Create database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Enhanced marketing database initialized!")

In [ ]:
# Create comprehensive campaigns table
cursor.execute('''
CREATE TABLE campaigns (
    campaign_id INTEGER PRIMARY KEY,
    campaign_name TEXT NOT NULL,
    channel TEXT NOT NULL,
    campaign_type TEXT,
    objective TEXT,
    targeting TEXT,
    start_date DATE,
    end_date DATE,
    budget REAL,
    status TEXT
)
''')

campaigns_data = [
    (1, 'Search_Brand_Q4_Desktop', 'Google', 'Search', 'Brand', 'Desktop', '2024-10-01', '2024-12-31', 50000, 'active'),
    (2, 'Search_Generic_Mobile_Q4', 'Google', 'Search', 'Acquisition', 'Mobile', '2024-10-01', '2024-12-31', 75000, 'active'),
    (3, 'FB_Prospecting_18-34', 'Meta', 'Social', 'Acquisition', '18-34', '2024-09-01', '2024-12-31', 60000, 'active'),
    (4, 'FB_Retargeting_AllAges', 'Meta', 'Social', 'Retention', 'All', '2024-09-01', '2024-12-31', 40000, 'active'),
    (5, 'Display_Remarketing_High_Intent', 'Google', 'Display', 'Retention', 'High_Intent', '2024-10-01', '2024-12-31', 30000, 'active'),
    (6, 'Instagram_Stories_Gen_Z', 'Meta', 'Social', 'Awareness', '18-24', '2024-10-15', '2024-12-31', 35000, 'active'),
    (7, 'LinkedIn_B2B_Decision_Makers', 'LinkedIn', 'Social', 'B2B', 'Directors+', '2024-11-01', '2024-12-31', 45000, 'active'),
    (8, 'TikTok_Awareness_Gen_Z', 'TikTok', 'Video', 'Awareness', '18-24', '2024-10-01', '2024-11-30', 25000, 'paused'),
    (9, 'YouTube_Video_35-54', 'Google', 'Video', 'Awareness', '35-54', '2024-10-01', '2024-12-31', 55000, 'active'),
    (10, 'Search_Competitor_Q4', 'Google', 'Search', 'Acquisition', 'All', '2024-11-01', '2024-12-31', 42000, 'active')
]

cursor.executemany('INSERT INTO campaigns VALUES (?,?,?,?,?,?,?,?,?,?)', campaigns_data)
print("✅ Campaigns table created")

In [ ]:
# Create daily performance data with realistic patterns
cursor.execute('''
CREATE TABLE daily_performance (
    performance_id INTEGER PRIMARY KEY,
    campaign_id INTEGER,
    date DATE,
    impressions INTEGER,
    clicks INTEGER,
    conversions INTEGER,
    cost REAL,
    revenue REAL,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id)
)
''')

# Generate 60 days of data per campaign
np.random.seed(42)
performance_data = []
perf_id = 1

for campaign_id in range(1, 11):
    for day in range(60):
        date = (datetime(2024, 10, 1) + timedelta(days=day)).strftime('%Y-%m-%d')
        
        # Different performance by campaign type
        if campaign_id in [1, 2, 10]:  # Search
            impressions = np.random.randint(3000, 8000)
            clicks = np.random.randint(150, 400)
            conversions = np.random.randint(8, 25)
            cost = np.random.uniform(200, 600)
            revenue = cost * np.random.uniform(3.5, 6.0)
        elif campaign_id in [3, 4, 6]:  # Social
            impressions = np.random.randint(15000, 40000)
            clicks = np.random.randint(500, 1500)
            conversions = np.random.randint(10, 35)
            cost = np.random.uniform(150, 450)
            revenue = cost * np.random.uniform(2.5, 4.5)
        else:  # Display/Video
            impressions = np.random.randint(25000, 60000)
            clicks = np.random.randint(300, 900)
            conversions = np.random.randint(5, 20)
            cost = np.random.uniform(100, 400)
            revenue = cost * np.random.uniform(2.0, 4.0)
        
        performance_data.append((perf_id, campaign_id, date, impressions, clicks, conversions, cost, revenue))
        perf_id += 1

cursor.executemany('INSERT INTO daily_performance VALUES (?,?,?,?,?,?,?,?)', performance_data)
print(f"✅ Daily performance table created with {len(performance_data)} rows")

In [ ]:
# Create customer journey / touchpoints table
cursor.execute('''
CREATE TABLE customer_touchpoints (
    touchpoint_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    campaign_id INTEGER,
    touchpoint_date DATE,
    touchpoint_type TEXT,
    touchpoint_order INTEGER,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id)
)
''')

# Sample multi-touch attribution data
touchpoints_data = [
    (1, 101, 6, '2024-10-15', 'impression', 1),
    (2, 101, 3, '2024-10-18', 'click', 2),
    (3, 101, 4, '2024-10-20', 'click', 3),
    (4, 101, 1, '2024-10-22', 'conversion', 4),
    (5, 102, 2, '2024-10-10', 'click', 1),
    (6, 102, 5, '2024-10-15', 'click', 2),
    (7, 102, 2, '2024-10-17', 'conversion', 3),
    (8, 103, 9, '2024-10-05', 'impression', 1),
    (9, 103, 3, '2024-10-12', 'click', 2),
    (10, 103, 4, '2024-10-14', 'conversion', 3)
]

cursor.executemany('INSERT INTO customer_touchpoints VALUES (?,?,?,?,?,?)', touchpoints_data)
print("✅ Customer touchpoints table created")

In [ ]:
conn.commit()
print("\n✅ Advanced marketing database setup complete!")

## 📅 Day 22: Advanced Window Functions (~60 min)

### Learning Objectives
- Use LAG and LEAD for time-based comparisons
- Apply NTILE for percentile rankings
- Calculate running totals and moving averages
- Use PERCENT_RANK for performance distribution

### The Business Problem
You need to:
- Compare this week's performance to last week
- Calculate 7-day moving averages
- Rank campaigns by efficiency percentiles
- Track cumulative spend pacing

### 📖 Concept: LAG and LEAD

Access previous (LAG) or next (LEAD) row values for comparisons.

In [ ]:
# Day-over-day comparison for one campaign
query = '''
SELECT 
    date,
    conversions,
    LAG(conversions, 1) OVER (ORDER BY date) as prev_day_conversions,
    conversions - LAG(conversions, 1) OVER (ORDER BY date) as day_over_day_change,
    ROUND(100.0 * (conversions - LAG(conversions, 1) OVER (ORDER BY date)) / 
          LAG(conversions, 1) OVER (ORDER BY date), 2) as pct_change
FROM daily_performance
WHERE campaign_id = 1
ORDER BY date
LIMIT 15;
'''

result = pd.read_sql_query(query, conn)
print("Day-over-Day Conversion Changes:")
print(result)

### 📖 Concept: Running Totals

Calculate cumulative values over time.

In [ ]:
# Budget pacing - track cumulative spend
query = '''
SELECT 
    c.campaign_name,
    c.budget,
    dp.date,
    dp.cost as daily_cost,
    SUM(dp.cost) OVER (PARTITION BY c.campaign_id ORDER BY dp.date) as cumulative_cost,
    ROUND(100.0 * SUM(dp.cost) OVER (PARTITION BY c.campaign_id ORDER BY dp.date) / c.budget, 2) as pct_budget_spent
FROM campaigns c
JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
WHERE c.campaign_id = 1
ORDER BY dp.date
LIMIT 20;
'''

result = pd.read_sql_query(query, conn)
print("Budget Pacing (Running Total):")
print(result)

### 📖 Concept: Moving Averages

Calculate rolling averages for trend smoothing.

In [ ]:
# 7-day moving average of conversions
query = '''
SELECT 
    date,
    conversions,
    ROUND(AVG(conversions) OVER (
        ORDER BY date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ), 2) as ma_7_day
FROM daily_performance
WHERE campaign_id = 1
ORDER BY date
LIMIT 20;
'''

result = pd.read_sql_query(query, conn)
print("7-Day Moving Average of Conversions:")
print(result)

### 📖 Concept: NTILE and PERCENT_RANK

Divide data into percentiles or calculate percentile ranks.

In [ ]:
# Rank campaigns by efficiency quartiles
query = '''
WITH campaign_metrics AS (
    SELECT 
        c.campaign_name,
        SUM(dp.revenue) / SUM(dp.cost) as roas
    FROM campaigns c
    JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
    GROUP BY c.campaign_id, c.campaign_name
)
SELECT 
    campaign_name,
    ROUND(roas, 2) as roas,
    NTILE(4) OVER (ORDER BY roas DESC) as performance_quartile,
    ROUND(PERCENT_RANK() OVER (ORDER BY roas DESC) * 100, 2) as percentile_rank
FROM campaign_metrics
ORDER BY roas DESC;
'''

result = pd.read_sql_query(query, conn)
print("Campaign Performance Quartiles:")
print(result)

### ✏️ Exercise 1: Week-over-Week Analysis

In [ ]:
# YOUR CODE HERE
# Calculate week-over-week performance for campaign_id = 2
# 1. Group daily_performance by week (use strftime('%W', date))
# 2. Calculate weekly conversions
# 3. Use LAG to get previous week's conversions
# 4. Calculate week-over-week change (absolute and percentage)
# 5. Show: week, weekly_conversions, prev_week, change, pct_change

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print(result)

### 🎯 Day 22 Mini-Project: Campaign Momentum Tracker

Build a momentum tracker showing which campaigns are trending up or down.

In [ ]:
# YOUR CODE HERE
# Create a campaign momentum report:
# 1. For each campaign, calculate ROAS for last 7 days vs previous 7 days
# 2. Calculate the change in ROAS
# 3. Classify momentum as:
#    - 'Accelerating' if ROAS improved > 10%
#    - 'Stable' if change between -10% and +10%
#    - 'Declining' if ROAS decreased > 10%
# 4. Rank campaigns by momentum
#
# Hint: You'll need to use date filtering and window functions

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Campaign Momentum Report:")
# print(result)

### 🎓 Day 22 Key Takeaways

✅ LAG/LEAD enable time-based comparisons  
✅ Running totals track cumulative metrics  
✅ Moving averages smooth noisy data  
✅ NTILE creates percentile buckets  
✅ PERCENT_RANK shows relative performance  

**Next:** Tomorrow we'll learn CTEs for cleaner, more readable queries!

---

## 📅 Day 23: CTEs and Temp Tables (~60 min)

### Learning Objectives
- Write Common Table Expressions (CTEs)
- Use multiple CTEs in one query
- Create recursive CTEs
- Use temporary tables for complex analysis

### The Business Problem
Complex queries become hard to read and maintain. CTEs make queries modular and understandable.

### 📖 Concept: Basic CTEs

CTEs (WITH clause) create named temporary result sets.

In [ ]:
# Without CTE (nested subquery - harder to read)
query = '''
SELECT 
    campaign_name,
    roas
FROM (
    SELECT 
        c.campaign_name,
        SUM(dp.revenue) / SUM(dp.cost) as roas
    FROM campaigns c
    JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
    GROUP BY c.campaign_id, c.campaign_name
) subquery
WHERE roas > 3.0
ORDER BY roas DESC;
'''

result = pd.read_sql_query(query, conn)
print("High ROAS Campaigns (without CTE):")
print(result)

In [ ]:
# With CTE (much cleaner!)
query = '''
WITH campaign_performance AS (
    SELECT 
        c.campaign_name,
        c.channel,
        SUM(dp.cost) as total_cost,
        SUM(dp.revenue) as total_revenue,
        SUM(dp.revenue) / SUM(dp.cost) as roas
    FROM campaigns c
    JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
    GROUP BY c.campaign_id, c.campaign_name, c.channel
)
SELECT 
    campaign_name,
    channel,
    ROUND(total_cost, 2) as total_cost,
    ROUND(total_revenue, 2) as total_revenue,
    ROUND(roas, 2) as roas
FROM campaign_performance
WHERE roas > 3.0
ORDER BY roas DESC;
'''

result = pd.read_sql_query(query, conn)
print("\nHigh ROAS Campaigns (with CTE):")
print(result)

### 📖 Concept: Multiple CTEs

Chain multiple CTEs for step-by-step logic.

In [ ]:
# Multi-stage analysis with multiple CTEs
query = '''
WITH campaign_metrics AS (
    -- Step 1: Calculate campaign-level metrics
    SELECT 
        c.campaign_id,
        c.campaign_name,
        c.channel,
        SUM(dp.cost) as total_cost,
        SUM(dp.revenue) as total_revenue,
        SUM(dp.conversions) as total_conversions
    FROM campaigns c
    JOIN daily_performance dp ON c.campaign_id = dp.campaign_id
    GROUP BY c.campaign_id, c.campaign_name, c.channel
),
campaign_efficiency AS (
    -- Step 2: Calculate efficiency metrics
    SELECT 
        *,
        total_revenue / total_cost as roas,
        total_cost / total_conversions as cpa
    FROM campaign_metrics
),
channel_benchmarks AS (
    -- Step 3: Calculate channel-level benchmarks
    SELECT 
        channel,
        AVG(roas) as avg_channel_roas,
        AVG(cpa) as avg_channel_cpa
    FROM campaign_efficiency
    GROUP BY channel
)
-- Step 4: Compare campaigns to channel benchmarks
SELECT 
    ce.campaign_name,
    ce.channel,
    ROUND(ce.roas, 2) as campaign_roas,
    ROUND(cb.avg_channel_roas, 2) as channel_avg_roas,
    CASE 
        WHEN ce.roas > cb.avg_channel_roas THEN 'Above Average'
        ELSE 'Below Average'
    END as performance_vs_channel
FROM campaign_efficiency ce
JOIN channel_benchmarks cb ON ce.channel = cb.channel
ORDER BY ce.channel, ce.roas DESC;
'''

result = pd.read_sql_query(query, conn)
print("Campaign vs Channel Benchmark:")
print(result)

### ✏️ Exercise 2: Build a Cohesive CTE Query

In [ ]:
# YOUR CODE HERE
# Create a query using multiple CTEs:
# 
# CTE 1: daily_roas - Calculate daily ROAS for each campaign
# CTE 2: campaign_avg - Calculate average daily ROAS per campaign
# CTE 3: volatile_days - Find days where ROAS was >20% different from campaign average
# 
# Final SELECT: Show campaign, date, daily ROAS, avg ROAS, deviation
# Only show volatile days

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Volatile Performance Days:")
# print(result)

### 🎯 Day 23 Mini-Project: Multi-Touch Attribution Analysis

Analyze customer journey touchpoints using CTEs.

In [ ]:
# YOUR CODE HERE
# Build a multi-touch attribution report using CTEs:
#
# CTE 1: customer_journeys - Get all touchpoints per customer
# CTE 2: journey_length - Count touchpoints per customer
# CTE 3: first_touch - Identify first touchpoint campaign
# CTE 4: last_touch - Identify last touchpoint campaign (conversion)
#
# Final SELECT: Show customer_id, journey length, first campaign, last campaign
# Show how many touchpoints between first and last

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Customer Journey Analysis:")
# print(result)

### 🎓 Day 23 Key Takeaways

✅ CTEs make complex queries readable  
✅ Multiple CTEs enable step-by-step logic  
✅ CTEs can reference earlier CTEs  
✅ Much cleaner than nested subqueries  
✅ Great for documenting query logic  

**Next:** Tomorrow we'll learn query optimization techniques!

---

## 📅 Day 24-27: Additional Advanced Topics

The following days cover:

**Day 24: Query Optimization**
- Understanding query execution plans
- Creating and using indexes
- Optimizing JOIN performance
- Avoiding common pitfalls

**Day 25: Date/Time Operations**
- Date arithmetic and intervals
- Extracting date parts (year, month, day, week)
- Cohort analysis by signup date
- Time-based segmentation

**Day 26: String Manipulation**
- Parsing campaign names
- LIKE and wildcards
- Regular expressions
- Extracting UTM parameters

**Day 27: Advanced Joins & Unions**
- CROSS JOIN for scenario analysis
- Self joins for comparisons
- UNION and UNION ALL
- Combining datasets from multiple sources

---

## 📅 Day 28: Week 4 Capstone - Complex Marketing Analytics (~60 min)

### Project: Executive Marketing Intelligence Report

**Scenario:**  
You're presenting to the CMO and CFO. They need a comprehensive, data-driven analysis of Q4 marketing performance with actionable insights for Q1 budget allocation.

**Requirements:**
Use advanced SQL techniques (window functions, CTEs, complex joins) to answer strategic questions.

**Deliverables:**
1. Campaign efficiency trends over time
2. Channel mix optimization recommendations
3. Customer acquisition cohort analysis
4. Multi-touch attribution insights
5. Budget reallocation simulation
6. Predictive performance indicators

### Challenge 1: Campaign Efficiency Trends

In [ ]:
# YOUR CODE HERE
# Create a weekly trend analysis showing:
# - Week number and date range
# - Weekly ROAS for each campaign
# - 4-week moving average ROAS
# - Week-over-week change in ROAS
# - Trend classification (improving/stable/declining)
#
# Use CTEs for clarity
# Use window functions for calculations
# Show only campaigns with declining trends in last 4 weeks

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Declining Campaign Trends:")
# print(result)

### Challenge 2: Channel Mix Optimization

In [ ]:
# YOUR CODE HERE
# Calculate optimal budget allocation:
# 1. Current spend share by channel (% of total budget)
# 2. Current revenue share by channel (% of total revenue)
# 3. ROAS by channel
# 4. Efficiency score: revenue_share / spend_share
#    (> 1.0 means channel generates more than its share)
# 5. Recommended budget adjustment
#
# Use CTEs to break down the logic
# Provide specific $ reallocation recommendations

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Channel Mix Optimization:")
# print(result)

### Challenge 3: Performance Percentile Analysis

In [ ]:
# YOUR CODE HERE
# Create a performance distribution analysis:
# 1. Calculate campaign-level ROAS
# 2. Assign each campaign to a decile (NTILE(10))
# 3. For each decile, show:
#    - Decile number (1 = best)
#    - Number of campaigns in decile
#    - Average ROAS in decile
#    - Total spend in decile
#    - % of total budget in decile
# 4. Identify concentration risk (top decile spending %)
#
# Use window functions and CTEs

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Performance Decile Analysis:")
# print(result)

### Challenge 4: Multi-Touch Attribution Report

In [ ]:
# YOUR CODE HERE
# Analyze customer touchpoint data:
# 1. First-touch attribution: Which campaigns started journeys?
# 2. Last-touch attribution: Which campaigns closed conversions?
# 3. Multi-touch: Which campaigns appeared in customer journeys?
# 4. For each campaign, calculate:
#    - First-touch attribution count
#    - Last-touch attribution count
#    - Assist count (appeared but not first/last)
#    - Total touchpoint appearances
#
# Use window functions to identify first/last in journey
# Compare first-touch vs last-touch attribution models

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Multi-Touch Attribution Analysis:")
# print(result)

### Challenge 5: Cohort Performance Analysis

In [ ]:
# YOUR CODE HERE
# Create a cohort analysis by campaign start month:
# 1. Group campaigns by start month
# 2. For each cohort (start month), calculate:
#    - Number of campaigns launched
#    - Average ROAS
#    - Total spend
#    - Total conversions
#    - Success rate (% of campaigns with ROAS > 3.0)
# 3. Compare cohorts to identify seasonal patterns
#
# Use date functions and CTEs

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Campaign Launch Cohort Analysis:")
# print(result)

### Challenge 6: Executive Summary Dashboard Query

In [ ]:
# YOUR CODE HERE
# Create a single comprehensive executive dashboard query using CTEs:
#
# Include:
# - Overall portfolio metrics (spend, revenue, ROAS, conversions)
# - Top 3 campaigns by ROAS with their metrics
# - Bottom 3 campaigns needing attention
# - Channel performance summary
# - Week-over-week trend (last week vs previous week)
# - Key recommendations based on data
#
# This should be production-ready SQL that could run daily
# Use CTEs to organize different sections
# Make output clear and actionable

query = '''

'''

# result = pd.read_sql_query(query, conn)
# print("Executive Dashboard:")
# print(result)

### 🎓 Week 4 Complete!

**Congratulations!** You've completed Week 4 of the Marketing Measurement Partner Academy.

**What You've Mastered:**
- ✅ Advanced window functions (LAG, LEAD, NTILE, PERCENT_RANK)
- ✅ Common Table Expressions (CTEs) for readable queries
- ✅ Query optimization techniques
- ✅ Date/time operations for cohort analysis
- ✅ String manipulation for campaign parsing
- ✅ Advanced joins and set operations
- ✅ Production-ready marketing analytics queries

**Your Advanced SQL Toolkit:**
You can now:
- Build complex, multi-stage analytical queries
- Perform sophisticated time-based analyses
- Calculate trends, moving averages, and cumulative metrics
- Compare performance across dimensions and time
- Create production-ready dashboards and reports
- Optimize queries for large datasets

**Next Week Preview:**
Week 5 will cover **Exploratory Data Analysis (EDA) Fundamentals**:
- Descriptive statistics
- Distribution analysis and outlier detection
- Correlation analysis
- Data profiling
- Segmentation and cohort analysis
- Campaign EDA project

**You're building professional-grade analytics skills!** 🚀

---